# Playing with TF, following examples on the web
Mostly from the project tensorflow_models

### A namedtuple can be used for storing hyper-params
It generates a class which its instances can return tuples, but fields can be referred by name. However, it would not work well with take_from_struct style params. So it would better be using a dictionary and fetching default values if a param doesnt exists.

In [12]:
from collections import namedtuple
HyperParams = namedtuple('HyperParams', 'lr, w_decay')
hp = HyperParams(lr=0.1, w_decay=1e-3)
print hp
a,b=hp
print a,b
print hp.lr

HyperParams(lr=0.1, w_decay=0.001)
0.1 0.001
0.1


## Reader and queues  for input pipeline


The TF pipeline supports reading data from multiple files (possibly with multiple threads), preprocessing it on the fly and feedining it as batches for training. This pipeline involves 3 queues that are piped.

#### Here is an illustration
![](https://www.tensorflow.org/versions/r0.10/images/AnimatedFileQueues.gif)

#### Here is a TL;DR from the TF (`QueueRunner`) tutorial
One set of threads generates filenames in a queue; a second set of threads read records from the files, processes them, and enqueues tensors on a second queue; a third set of threads dequeues these input records to construct batches and runs them through training operations.

#### In more details
Many of the `tf.train` functions listed above add `QueueRunner` objects to your graph. These require that you call `tf.train.start_queue_runners` before running any training or inference steps, or it will hang forever. This will start threads that run the input pipeline, filling the example queue so that the dequeue to get the examples will succeed. This is best combined with a `tf.train.Coordinator` to cleanly shut down these threads when there are errors. If you set a limit on the number of epochs, that will use an epoch counter that will need to be initialized. 